### Lab 3

In [1]:
#Import modules
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
titanic_df = pd.read_csv('./titanic.csv', sep=',')
titanic_df.head()